In [3]:
import requests
from lxml import html
import pandas as pd
import re
import json
import time
import random

In [4]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }

seller_page_headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }

seller_page_cookies = {
    'session-id' : '258-5942643-6431100',
    'i18n-prefs' : 'AED', 
    'ubid-acbae' : '259-8396416-4598938', 
    'session-token' : 'nzVs4t7q05a46vkJMc59s2WjN6JCZv5DI5x99owvyYTJcInd1BwEsBDNjk4T18Wenb786o+CXsciaSQPIxoWCBJXOdxxroZmpIQynzVhWKcl/Ru48Ox0Pp6IpUgK6BYd+CvAzlYMx8ZbhGaIsV4rtpc5lEaURwUEOd3wMDSvrp0HI2DRpyS6U8dPvrofvLSR',
    'csm-hit' : 'tb:TJAJVPCA5G2CQ9X1D585+s-TJAJVPCA5G2CQ9X1D585|1642767814793&t:1642767814794&adb:adblk_yes', 
    'session-id-time' : '2082787201l'
}

### Data from Product Page

In [7]:
## OOS product
url = 'https://www.amazon.com/Apple-iPhone-Pro-128GB-Gold/dp/B0BYLLTCMB/'
## Multivariant, Multiseller - SAMSUNG Galaxy S22 
url = 'https://www.amazon.com/SAMSUNG-Smartphone-Unlocked-Brightest-Processor/dp/B09MVZSW5V/'

In [8]:
res = requests.get(url, headers=headers)
tree = html.fromstring(res.text)
res.status_code

200

In [11]:
def product_name():
    product_name_xp1 = tree.xpath("//div[@id='centerCol']//span[@id='productTitle']/text()")
    
    product_name = ''
    if product_name_xp1:
        product_name = product_name_xp1[0].strip()
    
    return product_name

product_name()

'SAMSUNG Galaxy S22 Ultra Cell Phone, Factory Unlocked Android Smartphone, 128GB, 8K Camera & Video, Brightest Display Screen, S Pen, Long Battery Life, Fast 4nm Processor, US Version, Phantom Black'

In [10]:
def asin():
    asin_xp1 = tree.xpath("//input[@id='ASIN']/@value")

    asin = ""
    if asin_xp1:
        asin = asin_xp1[0].strip()
    return asin

asin()

''

In [9]:
def number_of_offers():
    no_of_offers_xp1 = tree.xpath("//div[@id='centerCol']//span[@data-action='show-all-offers-display']//span[contains(text(), 'New')]/text()")
    no_of_offers_xp2 = tree.xpath("//div[@id='rightCol']//div[@class='olp-text-box']//span[contains(text(), 'New')]/text()")
    
    no_of_offers = ''
    if no_of_offers_xp1:
        no_of_offers = re.sub("[^0-9]+", '', no_of_offers_xp1[0])
    elif no_of_offers_xp2:
        no_of_offers = re.sub("[^0-9]+", '', no_of_offers_xp2[0])
        
    return no_of_offers

print("Number of offers: ", number_of_offers())

Number of offers:  4


In [ ]:
def price_start_from():
    price_from_xp1 = tree.xpath("//div[@id='centerCol']//span[@data-action='show-all-offers-display']//span[contains(text(), 'New')]//following::span[@class='a-size-base a-color-price']/text()")
    price_from_xp2 = tree.xpath("//div[@id='rightCol']//div[@class='olp-text-box']//span[contains(text(), 'New')]//following::span[@class='a-size-base a-color-base']/text()")

    price_from = ''
    if price_from_xp1:
        price_from = re.sub("[^0-9.]+", '', price_from_xp1[0])
    elif price_from_xp2:
        price_from = re.sub("[^0-9.]+", '', price_from_xp2[0])
        
    return price_from

print("Price starts from: ", price_start_from())

### Seller URL

In [ ]:
def seller_url_link():
    home_url = "https://www.amazon.ae"
    asin_no = asin()
    seller_url =  home_url + "/gp/aod/ajax/ref=auto_load_aod?asin=" + asin_no + f"&pageno="
    return seller_url

seller_url_link()

## Data from Seller Page

In [ ]:
seller_url = seller_url_link()
seller_res = requests.get(seller_url, headers=headers)
seller_tree = html.fromstring(seller_res.content)

In [ ]:
# Seller Page data functions
def seller_name_sp_func(seller_row, seller_tree):
    # no_of_offers + 1 As we get 1 extra xpath in offer, if add not+2
    seller_name_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-soldBy']//a[@class='a-size-small a-link-normal'][1]/text()")
    seller_name_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-soldBy']//span[@class='a-size-small a-color-base'][1]/text()")
    # seller_row-1 : xpath row start from 1. 1st was skip i.e BuyBox
    seller_name_sp_xp3 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-soldBy']//a[@class='a-size-small a-link-normal']/text()")
    # for amazon.eg seller it wont show anchor tag
    seller_name_sp_xp4 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-soldBy']//span[@class='a-size-small a-color-base']/text()")

    seller_name_sp = ''
    if seller_name_sp_xp1 and seller_row == 1:
        seller_name_sp = seller_name_sp_xp1[0].strip()
    elif seller_name_sp_xp2 and seller_row == 1:
        seller_name_sp = seller_name_sp_xp2[0].strip()
    elif seller_name_sp_xp3:
        seller_name_sp = seller_name_sp_xp3[0].strip()
    elif seller_name_sp_xp4:
        seller_name_sp = seller_name_sp_xp4[0].strip()

    return seller_name_sp

def now_price_sp_func(seller_row, seller_tree):
    now_price_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//span[@class='a-price-whole'][1]/text()")
    now_price_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//span[@class='a-price-whole']/text()")

    now_price_sp = ''
    if now_price_sp_xp1 and seller_row == 1:
        now_price_sp = now_price_sp_xp1[0].strip()
    elif now_price_sp_xp2:
        now_price_sp = now_price_sp_xp2[0].strip()
        
    now_price_sp = re.sub('[^.0-9]+', '', now_price_sp)

    return now_price_sp

def seller_rating_sp_func(seller_row, seller_tree):
    seller_rating_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-seller-rating'][1]/child::i/@class")
    seller_rating_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-seller-rating'][1]/child::i/@class")

    seller_rating_sp = ''
    if seller_rating_sp_xp1 and seller_row == 1:
        seller_rating_sp = seller_rating_sp_xp1[0].split("mini-")[1].split()[0].replace("-", ".")
    elif seller_rating_sp_xp2:
        seller_rating_sp = seller_rating_sp_xp2[0].split("mini-")[1].split()[0].replace("-", ".")

    return seller_rating_sp

def shipping_cost_sp_func(seller_row, seller_tree):
    shipping_cost_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::a/text()")
    shipping_cost_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::a/text()")

    shipping_cost_sp = ''
    if shipping_cost_sp_xp1 and seller_row == 1:
        shipping_cost_sp = shipping_cost_sp_xp1[0].strip()
        if 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = 0
        elif not 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = re.sub("[^0-9.]", "", shipping_cost_sp)
            
    elif shipping_cost_sp_xp2:
        shipping_cost_sp = shipping_cost_sp_xp2[0].strip()
        if 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = 0
        elif not 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = re.sub("[^0-9.]", "", shipping_cost_sp)
    return shipping_cost_sp

def seller_ranking_sp_func(seller_row, seller_tree):
    return seller_row

def seller_review_count_sp_func(seller_row, seller_tree):
    seller_review_count_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//span[@id='aod-asin-reviews-count-title']/text()")
    seller_review_count_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//span[@id='seller-rating-count-{{iter}}']//span[contains(text(), 'rating')]/text()")

    seller_review_count_sp = ''
    if seller_review_count_sp_xp1 and seller_row == 1:
        seller_review_count_sp = seller_review_count_sp_xp1[0].strip()
        seller_review_count_sp = re.sub("[()]", '', seller_review_count_sp)
    elif seller_review_count_sp_xp2:
        seller_review_count_sp = seller_review_count_sp_xp2[0].strip()
        seller_review_count_sp = re.sub("[()]", '', seller_review_count_sp)

    return seller_review_count_sp

def shipping_days_text_sp_func(seller_row, seller_tree):
    shipping_days_text_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::b/text()")
    shipping_days_text_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::b/text()")

    shipping_days_text_sp = ''
    if shipping_days_text_sp_xp1 and seller_row == 1:
        shipping_days_text_sp = shipping_days_text_sp_xp1[0].strip()
    elif shipping_days_text_sp_xp2:
        shipping_days_text_sp = shipping_days_text_sp_xp2[0].strip()

    return shipping_days_text_sp

def product_condition_sp_func(seller_row, seller_tree):
    product_condition_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-heading']//h5/text()")
    product_condition_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-heading']//h5/text()")

    product_condition_sp = ''
    if product_condition_sp_xp1 and seller_row == 1: 
        product_condition_sp = product_condition_sp_xp1[0].strip()
    elif product_condition_sp_xp2:
        product_condition_sp = product_condition_sp_xp2[0].strip()

    return product_condition_sp


# Seller_name 2 diff values calculate
# Todo: Delivered by, Fulfilled by, Ships from
# Todo: Sold by
# Todo: prime_saving
# Todo: Fastest delivery Order within 7 hrs and 36 mins
# Tod: seller_positive_rating_txt - 47% positive over the last 12 months47% positive over the last 12 months

In [ ]:
def seller_data_crawl(seller_pages, seller_url_link):
    total_seller_pages = seller_pages()
    seller_url = seller_url_link()
    seller_data_sp = []
    seller_price_list_sp = []
    all_seller_pages_resp = ""

    # Seller pages loop
    # Todo: If one of the seller page response fails, then need to try 3 times hit with different proxy.
    for seller_page in range(1, total_seller_pages + 1):
        time.sleep(random.randint(3,5))
        print(f"seller url link{seller_page}: ", seller_url + str(seller_page))
        seller_resp = requests.get(seller_url + str(seller_page), headers=headers)
        
        # Append all seller pages response
        all_seller_pages_resp += seller_resp.text
        
        seller_tree = html.fromstring(seller_resp.text)
        
        # Todo: Delete Test Code
        # if seller_page == 2:
        #     seller_resp.status_code = 503
        # Todo: check if any seller page response is !== 200. Seller order changed - Major issue
        print("Seller page response: ", seller_resp.status_code)
        
        if seller_resp.status_code == 200:
            print(f"Fetching data for seller page {seller_page}")
            no_of_offers = number_of_offers()
            if no_of_offers != '':
                no_of_offers = int(no_of_offers)
                
            

            # Seller row in one page loop
            for seller_row in range(1, no_of_offers + 1):
                seller_name_sp = seller_name_sp_func(seller_row, seller_tree)
                now_price_sp = now_price_sp_func(seller_row, seller_tree)
                seller_rating_sp = seller_rating_sp_func(seller_row, seller_tree)
                shipping_cost_sp = shipping_cost_sp_func(seller_row, seller_tree)
                seller_ranking_sp = seller_ranking_sp_func(seller_row, seller_tree)
                seller_review_count_sp = seller_review_count_sp_func(seller_row, seller_tree)
                shipping_days_text_sp = shipping_days_text_sp_func(seller_row, seller_tree)
                product_condition_sp = product_condition_sp_func(seller_row, seller_tree)
                
                
                if seller_name_sp != "":
                    seller_data_sp.append({
                        'seller_name_sp': seller_name_sp,
                        'now_price_sp': now_price_sp,
                        'seller_rating_sp': seller_rating_sp,
                        'shipping_cost_sp': shipping_cost_sp,
                        'seller_ranking_sp': seller_ranking_sp,
                        'seller_review_count_sp': seller_review_count_sp,
                        'shipping_days_text_sp': shipping_days_text_sp,
                        'product_condition_sp': product_condition_sp,
                    })
           
        # seller resp != 200        
        else:
            pass

    print("Length seller data", len(seller_data_sp))
    return (seller_data_sp, all_seller_pages_resp)

seller_dataframe, all_seller_pages_resp = seller_data_crawl(seller_pages, seller_url_link)

### DataFrame

In [ ]:
df = pd.DataFrame(seller_dataframe)

In [12]:
df

NameError: name 'df' is not defined

# Testing

## Other functions